# Notebook to generate docs

In [26]:
import re
import importlib
import inspect

from pathlib import Path

class TEXT(str):
    def __repr__(self):
        return self
    

In [27]:
ROOTDIR = Path.cwd().parent
SRCDIR = ROOTDIR.joinpath("src").resolve(strict=True)


In [28]:
def list_modules(srcdir):
    srcdir = Path(srcdir)
    modules = []
    for path in srcdir.rglob("*.py"):
        path = path.relative_to(SRCDIR)
        name = path.as_posix().removesuffix(".py").replace("/", ".")
        name = name.removesuffix(".__init__")
        modules.append(name)
    return modules

list_modules(SRCDIR)

['mintalib.functions',
 'mintalib.reflib',
 'mintalib',
 'mintalib.indicators',
 'mintalib.model',
 'mintalib.builder',
 'mintalib.utils',
 'mintalib.testing',
 'mintalib.samples']

In [29]:
from mintalib.functions import EMA, DEMA

def trim_doc(doc):
    doc = re.sub("^ *\n", "", doc)
    doc = re.sub("\n+$", "\n", doc)
    doc = re.sub(" +$", "", doc, re.MULTILINE)

    indents = list(re.findall(r"^ *(?=\S)", doc, flags=re.MULTILINE))

    if indents and indents[0] == "":
        indents = indents[1:]

    if indents:
        dedent = min(indents)
        doc = re.sub(f"^{dedent}", "", doc, flags=re.MULTILINE)

    return doc


def convert_doc(doc):
    doc = trim_doc(doc)
    doc = re.sub(
        r"^[ ]+ (?= \w+ [ ]* [(:=] .+)", "- ", doc,
        flags = re.MULTILINE | re.VERBOSE)
    doc = re.sub(
        r"(?<= [:\n]\n) [ ]+", "+ ", doc,
        flags = re.MULTILINE | re.VERBOSE)
    return doc

doc = convert_doc(EMA.__doc__)

TEXT(doc)

Exponential Moving Average

Args:
- period (int) : time period, required
- adjust (bool) : whether to adjust weights, default False
        when true update ratio increases gradually (see formula)

Formula:
+ EMA is calculated as a recursive formula
    The standard formula is ema += alpha * (value - ema)
        with alpha = 2.0 / (period + 1.0)
    The adjusted formula is ema = num/div
        where num = value + rho * num, div = 1.0 + rho * div
        with rho = 1.0 - alpha

In [30]:
from collections import namedtuple

def get_ftype(func):
    if isinstance(func, type):
        return "class"

    if callable(func):
        if func.__module__.endswith("indicators"):
            return "indicator"
        return "function"

    return None


def annotate_parameter(param):
    if param.annotation is inspect._empty:
        if type(param.default) in (int, float, bool):
            return param.replace(annotation=type(param.default))

        if param.name in ("expr", "ma_type"):
            return param.replace(annotation=str)

        if param.name == "period":
            return param.replace(annotation=int)
    
    return param


def get_signature(func):
    parameters = inspect.signature(func).parameters.values()
    newparams = [annotate_parameter(p) for p in parameters]
    signature = inspect.Signature(newparams)
    result = str(signature)
    result = re.sub("(?<=: )'(\w+)'", r"\1", result)
    return result 

def make_doc(module_name, func_names=None, *, strict: bool=True):
    output = ""

    module = importlib.import_module(module_name, package=None)

    output += f"# `{module_name}` module\n\n"

    if module.__doc__:
        doc = trim_doc(module.__doc__)
        output += doc + "\n\n"

    func_names = [k for k, v in vars(module).items() if callable(v) and not k.startswith("_")]

    for name in  func_names:
        func = getattr(module, name)
        ftype = get_ftype(func)
        if not ftype:
            continue
        if strict and func.__module__ != module_name:
            continue
        signature = get_signature(func)
        output += f"## `{name}` {ftype}\n\n"
        output += f"```python\n{name}{signature}\n```\n\n"
        if func.__doc__:
            doc = convert_doc(func.__doc__)
            output += doc
        output += "\n"

    return output

output = make_doc("mintalib.functions")

TEXT(output)


# `mintalib.functions` module

Calculation functions for technical analysis indicators.

The function names are all lower case and may conflict with standard functions,
so the best way to use this module is to alias it to a short name
like `ta` and access all functions as attributes.

The first parameter `series` or `prices` indicates whether the function
accepts a single series or a prices dataframe.

Functions that accept a series usually have an optional parameter `item`
to specify which column to use if the input is a dataframe.

All functions wrap their output to match the type of their input.

In particular the result of a function applied to a pandas series or dataframes
will have the same index as the input.


## `price` function

```python
price(prices, item: str = None)
```

Generic Price 
   
   Args:
- item (str) : one of 'open', 'high', 'low', 'close',
           'avg', 'mid', 'typ', 'wcl' defaults to 'close'

## `avgprice` function

```python
avgprice(prices)
```

Average

In [31]:
def save_doc(module, func_names=None, *, strict: bool = True):
    docs = ROOTDIR / "docs"
    docs.mkdir(parents=True, exist_ok=True)

    output = make_doc(
        module,
        func_names=func_names,
        strict=strict
    )

    mdfile = docs / f"{module}.md"
    mdfile.write_text(output)

    return TEXT(output)


In [32]:
save_doc("mintalib.functions")

# `mintalib.functions` module

Calculation functions for technical analysis indicators.

The function names are all lower case and may conflict with standard functions,
so the best way to use this module is to alias it to a short name
like `ta` and access all functions as attributes.

The first parameter `series` or `prices` indicates whether the function
accepts a single series or a prices dataframe.

Functions that accept a series usually have an optional parameter `item`
to specify which column to use if the input is a dataframe.

All functions wrap their output to match the type of their input.

In particular the result of a function applied to a pandas series or dataframes
will have the same index as the input.


## `price` function

```python
price(prices, item: str = None)
```

Generic Price 
   
   Args:
- item (str) : one of 'open', 'high', 'low', 'close',
           'avg', 'mid', 'typ', 'wcl' defaults to 'close'

## `avgprice` function

```python
avgprice(prices)
```

Average

In [33]:

save_doc("mintalib.indicators")

# `mintalib.indicators` module

Factory functions for technical analysis indicators.

Indicator factory names are all upper case.

Indicators offer a composable interface where a calculation routine
is bound together with its calculation parameters.

An indicator object is a callable that can be applied to prices or series data.

Indicators can be chained with the `@` operator as in `ROC(1) @ SMA(20)`.

The `@` operator can also be used to apply an indicator to its parameter.

So for example `SMA(50) @ prices` can be used to compute the 50 period simple moving average on `prices`,
instead of the more verbose `SMA(50)(prices)`.


## `PRICE` indicator

```python
PRICE(item: str = None)
```

Generic Price 
   
   Args:
- item (str) : one of 'open', 'high', 'low', 'close',
           'avg', 'mid', 'typ', 'wcl' defaults to 'close'

## `AVGPRICE` indicator

```python
AVGPRICE()
```

Average Price

Value of (open + high + low + close) / 4

## `TYPPRICE` indicator

```python
TYPPRICE()
```

Ty

In [34]:
save_doc("mintalib.core", func_names = str.islower, strict=True)

# `mintalib.core` module

Calculation routines implemented in cython.

Routines are typically named `calc_` followed by an indicator name all in lower caps as in `calc_sma`.

The first parameter `series` or `prices` indicates whether the calculation accepts a single series or a prices dataframe.

A `prices` dataframe should contain the columns `open`, `high`, `low`, `close` and optionally `volume` all in **lower case**.

The `wrap` parameter dictates whether to wrap the calculation result to match the type of the inputs.


## `check_size` function

```python
check_size(xs, *others)
```

check all series have the same size and return the size
## `column_accessor` function

```python
column_accessor(data)
```

column accessor if applicable
## `get_series` function

```python
get_series(data, item: str = None, *, default_item: str = 'close')
```

get series from prices or series data
## `with_metadata` function

```python
with_metadata(*, same_scale: bool = None)
```

update function with

In [35]:
from mintalib.functions import sma, get_series

In [36]:
sma.__module__

'mintalib.functions'

In [37]:
get_series.__module__

'mintalib.core'

In [38]:
vars()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import re\nimport importlib\nimport inspect\n\nfrom pathlib import Path\n\nclass TEXT(str):\n    def __repr__(self):\n        return self\n    ',
  'ROOTDIR = Path.cwd().parent\nSRCDIR = ROOTDIR.joinpath("src").resolve(strict=True)',
  'def list_modules(srcdir):\n    srcdir = Path(srcdir)\n    modules = []\n    for path in srcdir.rglob("*.py"):\n        path = path.relative_to(SRCDIR)\n        name = path.as_posix().removesuffix(".py").replace("/", ".")\n        name = name.removesuffix(".__init__")\n        modules.append(name)\n    return modules\n\nlist_modules(SRCDIR)',
  'from mintalib.functions import EMA, DEMA\n\ndef trim_doc(doc):\n    doc = re.sub("^ *\\n", "", doc)\n    doc = re.sub("\\n+$", "\\n", do

In [39]:
from mintalib import core

In [40]:
vars(core)

{'__name__': 'mintalib.core',
 '__doc__': '\nCalculation routines implemented in cython.\n\nRoutines are typically named `calc_` followed by an indicator name all in lower caps as in `calc_sma`.\n\nThe first parameter `series` or `prices` indicates whether the calculation accepts a single series or a prices dataframe.\n\nA `prices` dataframe should contain the columns `open`, `high`, `low`, `close` and optionally `volume` all in **lower case**.\n\nThe `wrap` parameter dictates whether to wrap the calculation result to match the type of the inputs.\n',
 '__package__': 'mintalib',
 '__loader__': <_frozen_importlib_external.ExtensionFileLoader at 0x107e3b6d0>,
 '__spec__': ModuleSpec(name='mintalib.core', loader=<_frozen_importlib_external.ExtensionFileLoader object at 0x107e3b6d0>, origin='/Users/frederic/Projects/mintalib/src/mintalib/core.cpython-39-darwin.so'),
 '__file__': '/Users/frederic/Projects/mintalib/src/mintalib/core.cpython-39-darwin.so',
 '__builtins__': <module 'builtins' 